# Inserting multiple documents

In [1]:
import pymongo
cli = pymongo.MongoClient()
db = cli.test

In [2]:
videos = [
    {'title': 'Python Design Patterns'},
    {'title': 'Bootstrap 4 Projects'} ,
    {'title': 'Ethical Hacking for Beginners'},
    {'title': 'Expert Ruby on Rails 5'}
]

In [3]:
db.packt_videos.drop()

In [4]:
result = db.packt_videos.insert_many(videos)

In [5]:
result

In [6]:
result.inserted_ids

[ObjectId('58deb2e64e92acbdd4caf314'),
 ObjectId('58deb2e64e92acbdd4caf315'),
 ObjectId('58deb2e64e92acbdd4caf316'),
 ObjectId('58deb2e64e92acbdd4caf317')]

In [7]:
list(db.packt_videos.find())

[{'_id': ObjectId('58deb2e64e92acbdd4caf314'),
  'title': 'Python Design Patterns'},
 {'_id': ObjectId('58deb2e64e92acbdd4caf315'),
  'title': 'Bootstrap 4 Projects'},
 {'_id': ObjectId('58deb2e64e92acbdd4caf316'),
  'title': 'Ethical Hacking for Beginners'},
 {'_id': ObjectId('58deb2e64e92acbdd4caf317'),
  'title': 'Expert Ruby on Rails 5'}]

# Compound Bulk Operations

In [8]:
videos = [
    {'title': 'Python Design Patterns', 'authors': ['Gergo Bogdan']},
    {'title': 'Bootstrap 4 Projects', 'authors': ['Aaron Vanston']}, 
    {'title': 'Expert Ruby on Rails 5', 'authors': ['Fred Heath']},
    {'title': 'OpenFrame works Interactivity', 'authors': ['Denis Perevalov', 'Igor Tatarnikov']}
]

In [9]:
bulk = db.packt_videos.initialize_ordered_bulk_op()

In [10]:
for video in videos:
    op = bulk.find({'title': video['title']})
    op = op.upsert()
    op = op.update({'$addToSet': {'authors': {'$each': video['authors']}}})
result = bulk.execute()
result

{'nInserted': 0,
 'nMatched': 3,
 'nModified': 3,
 'nRemoved': 0,
 'nUpserted': 1,
 'upserted': [{'_id': ObjectId('58deb2e78b8e4054555bfc4f'), 'index': 3}],
 'writeConcernErrors': [],
 'writeErrors': []}

In [11]:
list(db.packt_videos.find({}, {'_id': 0}))

[{'authors': ['Gergo Bogdan'], 'title': 'Python Design Patterns'},
 {'authors': ['Aaron Vanston'], 'title': 'Bootstrap 4 Projects'},
 {'title': 'Ethical Hacking for Beginners'},
 {'authors': ['Fred Heath'], 'title': 'Expert Ruby on Rails 5'},
 {'authors': ['Denis Perevalov', 'Igor Tatarnikov'],
  'title': 'OpenFrame works Interactivity'}]

# Handling errors

In [12]:
doc = db.packt_videos.find_one()

In [13]:
from pprint import pprint

bulk = db.packt_videos.initialize_ordered_bulk_op()
bulk.insert(doc)
try:
    result = bulk.execute()
    print('success')
except pymongo.errors.BulkWriteError as bwe:
    pprint(bwe.details)

{'nInserted': 0,
 'nMatched': 0,
 'nModified': 0,
 'nRemoved': 0,
 'nUpserted': 0,
 'upserted': [],
 'writeConcernErrors': [],
 'writeErrors': [{'code': 11000,
                  'errmsg': 'E11000 duplicate key error collection: '
                            'test.packt_videos index: _id_ dup key: { : '
                            "ObjectId('58deb2e64e92acbdd4caf314') }",
                  'index': 0,
                  'op': {'_id': ObjectId('58deb2e64e92acbdd4caf314'),
                         'authors': ['Gergo Bogdan'],
                         'title': 'Python Design Patterns'}}]}


In [14]:
bulk = db.packt_videos.initialize_unordered_bulk_op()

In [15]:
for video in videos:
    op = bulk.find({'title': video['title']})
    op = op.upsert()
    op = op.update({'$addToSet': {'authors': {'$each': video['authors']}}})
result = bulk.execute()
result

{'nInserted': 0,
 'nMatched': 4,
 'nModified': 0,
 'nRemoved': 0,
 'nUpserted': 0,
 'upserted': [],
 'writeConcernErrors': [],
 'writeErrors': []}